In [1]:
# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import tensorflow as tf
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard,CSVLogger
from tensorflow.keras.optimizers import *
from keras.models import Model
# from keras.layers import Dense
from tensorflow.keras.layers import *
# from keras.layers import Flatten
import keras
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
df = pd.read_csv(r'C:\Users\mcu\Desktop\Label.csv')  

In [ ]:
#############detect face location#####################################
filenames = []
for i in range(9760):
    name = "C:\\Users\\mcu\\Desktop\\data\\" + str(i) + ".jpg"
    filenames.append(name)

file_2name = 0
detector = MTCNN()

for filename in filenames:        
    print(filename)
    try:
        pixels = pyplot.imread(filename)
        results = detector.detect_faces(pixels)
        # extract the bounding box from the first face
        x1, y1, width, height = results[0]['box']
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize((224, 224))
        name_2 = "C:\\Users\\mcu\\Desktop\\face\\" + str(file_2name) + ".jpg"
        image.save(name_2, 'JPEG')
        file_2name = file_2name + 1
    except:
        print("---------------")
        continue



In [3]:
# #read the face file and do preprocess
faces = []
for i in range(9761):
    name_2 = "C:\\Users\\mcu\\Desktop\\face\\" + str(i) + ".jpg"
    pixels = pyplot.imread(name_2)
    pixels = asarray(pixels, 'float32')
    faces.append(pixels)
samples = asarray(faces, 'float32')
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(faces, version=2)

In [4]:
 # create a vggface model transfer learning
model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

for layer in model.layers:
    layer.trainable = False
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(512, activation='relu')(flat1)
output = Dense(8, activation='softmax')(class1)
model = Model(inputs=model.inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
______________________________________________________________________________________________

In [6]:
from keras.utils.np_utils import to_categorical
categorical_labels = to_categorical(df['Target'], num_classes=8)

In [8]:
checkpoint = ModelCheckpoint(filepath='C:\\Users\\mcu\\Desktop\\model_best.h5', verbose=1, save_best_only=False)
earlystop = EarlyStopping( patience=10, verbose=1, mode='auto')
callbacks_list = [checkpoint, earlystop]
model.fit(x=samples, y=categorical_labels,batch_size=500,epochs=20,callbacks = [callbacks_list],validation_split = 0.1,shuffle=True)

Epoch 1/20
18/18 [==============================] - 168s 9s/step - loss: 1.2639 - accuracy: 0.8049 - val_loss: 4.3110 - val_accuracy: 0.4135

Epoch 00001: saving model to C:\Users\mcu\Desktop\model_best.h5
Epoch 2/20
18/18 [==============================] - 167s 9s/step - loss: 0.0133 - accuracy: 0.9979 - val_loss: 0.0986 - val_accuracy: 0.9908

Epoch 00002: saving model to C:\Users\mcu\Desktop\model_best.h5
Epoch 3/20
18/18 [==============================] - 167s 9s/step - loss: 0.0046 - accuracy: 0.9998 - val_loss: 0.1625 - val_accuracy: 0.9826

Epoch 00003: saving model to C:\Users\mcu\Desktop\model_best.h5
Epoch 4/20
18/18 [==============================] - 168s 9s/step - loss: 0.0129 - accuracy: 0.9996 - val_loss: 0.2297 - val_accuracy: 0.9775

Epoch 00004: saving model to C:\Users\mcu\Desktop\model_best.h5
Epoch 5/20
18/18 [==============================] - 166s 9s/step - loss: 0.0020 - accuracy: 0.9999 - val_loss: 0.2761 - val_accuracy: 0.9713

Epoch 00005: saving model to C:\Us

In [ ]:
yhat = model.predict(samples)

In [ ]:
yhat[1]

In [ ]:
import numpy as np
np.argmax( yhat ,axis=1)